In [ ]:
import json
import glob

file_paths = glob.glob("/mnt/ShareDrive/shared/Spotify Extended Streaming History/Streaming_History_Audio_*.json")

all_streams = []
for path in file_paths:
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        all_streams.extend(data)


In [ ]:
print(f"Loaded {len(all_streams)} total plays.")
print(all_streams[0])  

Loaded 159816 total plays.
{'ts': '2021-08-14T20:18:26Z', 'username': 'rt47etgc6xpwhhhb8575rth83', 'platform': 'Android-tablet OS 11 API 30 (SAMSUNG, SM-T870)', 'ms_played': 185351, 'conn_country': 'IN', 'ip_addr_decrypted': '106.207.22.18', 'user_agent_decrypted': 'unknown', 'master_metadata_track_name': 'Cold Water', 'master_metadata_album_artist_name': 'Major Lazer', 'master_metadata_album_album_name': 'Major Lazer Essentials', 'spotify_track_uri': 'spotify:track:6xsGllAEwkmF4SSDwdpmMt', 'episode_name': None, 'episode_show_name': None, 'spotify_episode_uri': None, 'reason_start': 'trackdone', 'reason_end': 'trackdone', 'shuffle': False, 'skipped': None, 'offline': False, 'offline_timestamp': 1628972119379, 'incognito_mode': False}


In [ ]:
from collections import defaultdict
from datetime import datetime
import json
import glob

yearly_data = defaultdict(lambda: defaultdict(list))

for stream in all_streams:
    ts = stream.get("ts")
    uri = stream.get("spotify_track_uri")
    if ts and uri:
        year = datetime.fromisoformat(ts.replace("Z", "")).year
        yearly_data[year][uri].append(ts)

filtered_sorted_tracks_by_year = {}

for year, track_dict in yearly_data.items():
    filtered_tracks = {
        uri: max(timestamps)
        for uri, timestamps in track_dict.items()
        if len(timestamps) >= 5
    }
    sorted_uris = sorted(filtered_tracks, key=filtered_tracks.get, reverse=True)
    filtered_sorted_tracks_by_year[year] = sorted_uris

for year, uris in filtered_sorted_tracks_by_year.items():
    print(f"{year} — {len(uris)} tracks")
    print(uris[:5])

2021 — 1724 tracks
['spotify:track:4RWzi7WNbW3H1Rr0aE9oPl', 'spotify:track:1Mfl0t1hvP1v6EJL1yIuWf', 'spotify:track:7rRMmhC027ixjjXzuZYoMv', 'spotify:track:47sGeWDJKLOGSsixHhYJZ8', 'spotify:track:6hFi0gXP8KItwMqfBgf44b']
2023 — 1563 tracks
['spotify:track:4LyO99BhEMAyVGe4J5genE', 'spotify:track:2BjRhZ2IJmsO1GRquNldsP', 'spotify:track:3zFy1R0QhiT3YMXTxV0KRd', 'spotify:track:0Hq2ZhF4uuDJheQBfsXU4p', 'spotify:track:5LLa7Xb6xfRVpRXgjyevvM']
2022 — 1753 tracks
['spotify:track:2t0wwvR15fc3K1ey8OiOaN', 'spotify:track:3nfosq8fXaM506M6L52kjI', 'spotify:track:2iLkTMUJ2MCsry5yYiafCg', 'spotify:track:2pAD0PKMUMtxlMvz33tYzB', 'spotify:track:3hRV0jL3vUpRrcy398teAU']
2020 — 851 tracks
['spotify:track:5q35GiekvBr6oAsCsW8YlR', 'spotify:track:44gRhRi2OhEf7moAUj6MD1', 'spotify:track:5rBtFEgasUy6eHRJcQ8Qlh', 'spotify:track:2B5ub9OB8rZj5PqvsAwrKo', 'spotify:track:09PGubKAMryhOWv1LHpCYz']
2019 — 144 tracks
['spotify:track:0V0bWil6rS8w4Xbn6eGwCQ', 'spotify:track:4TnjEaWOeW0eKTKIEvJyCa', 'spotify:track:3N304YQ

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id="YOUR_CLIENT_ID",
    client_secret="YOUR_CLIENT_SECRET",
    redirect_uri="http://localhost:8888/callback",
    scope="playlist-modify-public"
))

user_id = sp.current_user()["id"]

for year, uris in filtered_sorted_tracks_by_year.items():
    name = f"Top Songs {year} (5+ Plays)"
    playlist = sp.user_playlist_create(user=user_id, name=name, public=True)
    playlist_id = playlist['id']
    
    for i in range(0, len(uris), 100):
        sp.playlist_add_items(playlist_id, uris[i:i+100])
    print(f"Created playlist: {name} with {len(uris)} tracks")
